In [1]:
### 1. Data Preparation
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split

# Load dataset (replace with your data path)
url = "Steel_industry_data.csv"
df = pd.read_csv(url)


In [3]:
df.head()

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [4]:
# Convert categorical variables
encoder = OrdinalEncoder()
df[['WeekStatus', 'Load_Type']] = encoder.fit_transform(df[['WeekStatus', 'Load_Type']])


In [5]:
df.head()

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,0.0,Monday,0.0
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,0.0,Monday,0.0
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,0.0,Monday,0.0
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,0.0,Monday,0.0
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,0.0,Monday,0.0


In [9]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

In [11]:

df = df.resample('H', on='date').agg({
    'Usage_kWh': 'mean',
    'Lagging_Current_Reactive.Power_kVarh': 'mean',  # Corrected column name
    'Leading_Current_Reactive_Power_kVarh': 'mean',
    'CO2(tCO2)': 'mean',
    'Lagging_Current_Power_Factor': 'mean',
    'Leading_Current_Power_Factor': 'mean',
    'NSM': 'first',
    'WeekStatus': 'first',
    'Load_Type': 'first'
})


C:\Users\dinit\AppData\Local\Temp\ipykernel_956\3086138976.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H', on='date').agg({


In [12]:

# Normalization
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [13]:

# Sliding window transformation
def create_sliding_windows(data, window_size=3):
    X, y = [], []
    for i in range(len(data)-window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size, 0])  # First column is target (Usage_kWh)
    return np.array(X), np.array(y)


In [14]:
window_size = 16  # Best from paper
X, y = create_sliding_windows(scaled_data, window_size)

In [15]:
# Train-test split (82-18)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, shuffle=False)

In [16]:
### 2. Model Architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout

In [18]:
def build_lstm_model(model_type='single', input_shape=(16, 9)):
    model = Sequential()
    
    if model_type == 'single':
        model.add(LSTM(64, input_shape=input_shape))
    elif model_type == 'double':
        model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
        model.add(LSTM(64))
    elif model_type == 'bidirectional':
        model.add(Bidirectional(LSTM(64), input_shape=input_shape))
    
    model.add(Dropout(0.1))
    model.add(Dense(1))
    return model

In [19]:

# Initialize best model (double-layer)
model = build_lstm_model('double', (window_size, X_train.shape[2]))

c:\Users\dinit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:

### 3. Model Training
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [21]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - loss: 0.0359 - mae: 0.1458 - val_loss: 0.0198 - val_mae: 0.0999
Epoch 2/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.0227 - mae: 0.1100 - val_loss: 0.0157 - val_mae: 0.0863
Epoch 3/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.0208 - mae: 0.1007 - val_loss: 0.0149 - val_mae: 0.0841
Epoch 4/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0167 - mae: 0.0889 - val_loss: 0.0119 - val_mae: 0.0763
Epoch 5/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0131 - mae: 0.0799 - val_loss: 0.0085 - val_mae: 0.0613
Epoch 6/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0108 - mae: 0.0696 - val_loss: 0.0079 - val_mae: 0.0562
Epoch 7/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 0.0098 - mae: 0.0652 - val_loss: 0.0074 - val_mae: 0.0542
Epoch 8/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0097 - mae: 0.0647 - val_loss: 0.0069 - val_mae: 0.0503
Epoch 9/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms

In [22]:
### 4. Evaluation Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [25]:
def willmott_index(y_true, y_pred):
    numerator = np.sum((y_pred - y_true)**2)
    denominator = np.sum((np.abs(y_pred - np.mean(y_true)) + 
                        np.abs(y_true - np.mean(y_true)))**2)
    return 1 - (numerator / denominator)

In [26]:
def evaluate_model(model, X_test, y_test):
	from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

	# Predict on test data
	y_pred = model.predict(X_test)

	# Calculate metrics
	mse = mean_squared_error(y_test, y_pred)
	mae = mean_absolute_error(y_test, y_pred)
	r2 = r2_score(y_test, y_pred)

	return {"MSE": mse, "MAE": mae, "R2": r2}
metrics = evaluate_model(model, X_test, y_test)


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


In [36]:
metrics

{'MSE': 0.00567912537267371,
 'MAE': 0.043865385186735446,
 'R2': 0.8810534898720461}

In [27]:
### 5. XAI with SHAP
import shap

In [35]:
# Sample background data for SHAP
background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]

In [41]:
import tensorflow as tf
import shap

# 1. Install required package
# Run in terminal: pip install tf_keras

# 2. Import from standalone tf_keras
from tf_keras.backend import set_session

# 3. Configure session
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
set_session(session)

# 4. SHAP implementation
explainer = shap.DeepExplainer(model, background, session=session)
shap_values = explainer.shap_values(X_test[:100])


ModuleNotFoundError: No module named 'tf_keras'

In [ ]:

# Plot feature importance
shap.summary_plot(shap_values, X_test[:100], feature_names=df.columns)

### 6. Prediction Visualization
import matplotlib.pyplot as plt

# Inverse transform predictions
y_pred = model.predict(X_test)
y_test_inv = scaler.inverse_transform(np.concatenate((
    y_test.reshape(-1,1), 
    np.zeros((len(y_test), scaled_data.shape[1]-1))
), axis=1))[:,0]

y_pred_inv = scaler.inverse_transform(np.concatenate((
    y_pred.reshape(-1,1), 
    np.zeros((len(y_pred), scaled_data.shape[1]-1))
), axis=1))[:,0]

# Plot results
plt.figure(figsize=(15,6))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted', alpha=0.7)
plt.title('Energy Consumption Forecasting')
plt.ylabel('kWh')
plt.xlabel('Time Steps')
plt.legend()
plt.show()

### 7. Model Saving
model.save('energy_forecast_lstm.h5')
